<a href="https://colab.research.google.com/github/darshit8141/capstone-project-1/blob/main/capstone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project By Letsupgrade**

## **Problem Statement**

## City Scan Image Classification

### The dataset contains 1252 CT scans that are positive for SARS-COV-2 infection(Covid-19) and 1230 CT scans for patients non- infected by SARS-COV-2, 2482 CT scans in total. These data has been collected from real patients in hospitals from Sau Paulo,Brazil. The aim of this dataset is to encourage the research and development of artificial intelligent methods which are able to identify if a person is infected by SARS-COV-2 through the analysis of his/her CT scans.

## **Importing useful libraries for our model**

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split



In [ ]:
# Loading data
dataset = r"E:\capstone"

In [ ]:
# Setting image size and batch size
img_size = (180,180)
batch_size = 32

## **2-The images are in different sizes so you have to take a fixed size on which you have to work**

## **3-Do Dataaugmentation on it, mention atleast 5 args inside it.**

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the list of image files
covid_images= [os.path.join(dataset,'COVID', img) for image in os.listdir(os.path.join(dataset,'COVID',))]
covid_images= [os.path.join(dataset,'non-COVID', img) for image in os.listdir(os.path.join(dataset,'non-COVID',))]


In [ ]:
# Split the data into train and validation sets
covid_train, covid_val = train_test_split(covid_images,test_size= 0.2, random_state=3)
non_covid_train, non_covid_val = train_test_split(non_covid_images,test_size= 0.2, random_state=3)


In [ ]:
train_df = pd.DataFrame({'filename': covid_train + non_covid_train, 'class': ['covid']*len(covid_train) + ['non-covid']*len(non_covid_train) })
val_df = pd.DataFrame({'filename': covid_val + non_covid_val, 'class': ['covid']*len(covid_val) + ['non-covid']*len(non_covid_val) })

## **Here in the place of 'Target_size' we are resizing the images that can fix all the image sizes into the same sizes**

In [ ]:
# Create train and validation generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col= 'filename',
    y_col= 'class',
    target_size = img_size  # here in the place of target_size we are resizing the images that can fix all the image sizes into the same sizes
    batch_size = batch_size
    class_mode = 'binary',
    )


val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col= 'filename',
    y_col= 'class',
    target_size = img_size  # here in the place of target_size we are resizing the images that can fix all the image sizes into the same sizes
    batch_size = batch_size
    class_mode = 'binary',
    )

Found 1983 validated image filenames belonging to 2-classes.

  Found 497 validated image filenames belonging to 2-classes.

In [ ]:
# Importing libraries which is used to build architecture
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, callbacks

### **4-Train model by using only resnet from resnet you can pick any layer model like - ResNet-18, ResNet-34, ResNet-50, ResNet-101 ..**

### **Here I choose ResNet-50.**

In [ ]:
res50 = ResNet50(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)

In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
from keras.layers import Dense, Conv2D, MaPool2D, Flatten, Droupout, BatchNormalization

In [ ]:
# Fully connected layers
flatten = Flatten()(res50.output)
dense = Dense(512, activation = 'relu')(flatten)
dense = Droupout(o.5)(dense)
dense = Dense(128, activation = 'relu')(dense)
dense = Droupout(o.3)(dense)

# Output layer
prediction = Dense(1, activation = 'sigmoid', dense)


In [ ]:
from keras.models import Sequential, Model

In [ ]:
model1 = Model(inputs = res50.input, outputs = prediction)

In [ ]:
# Summary of model
model1.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 3, # Numbers of epochs with no improvement after which training will be stopped
    restore_best_weights = True
)

In [ ]:
history1 = model1.fit(
    train_generator,
    epochs = 15,
    validation_data = validation_generator,
    callbacks = [early_stopping],
)

### **5-Do prediction and mention multiple perfomance metrics**

In [ ]:
test_results = model1.evaluate(validation_generator)
print(f:"Test Loss : {test_results[0]} Test Accuracy :{test_results[1]}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
y_true = validation_generator.classes # True classes
y_pred_prob = model1.predict(validation_generator) # Predicted probablities

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int) # Adjust threshold as needed

# Calculate metrics
accuracy = acrracy_score(y_true , y_predy)
precision = precision_score(y_true , y_predy)
recall = recall_score(y_true , y_predy)
f1 = f1_score(y_true , y_predy)

# Confusion matrix
conf_matrix = confusion_matrix(y_true , y_pred)


print("validation Accuracy :", accuracy)
print("validation Precision :", precision)
print("validation Recall :", recall)
print("validation f1_score :", f1)

print("\nConfusion Matrix :")
print(conf_matrix)

## **Here train accuracy = acurracy: 0.851**

## **Here train accuracy = test acc: 0.8511**

### **After perfoming many attempts our model works with 85% accuracy it is fine to perform image classification.**

## **Doing Prediction**

### **Here I created a function that can follow the model architecture and predicts whether the CT scans belongs to Covid and Non-covid**

In [ ]:
import cv2
import matplotlib.pyplot as plt
def cov():

  input_image_path = input()
  input_image = cv2.imread(input_image_path)

  plt.imshow(cv2.cvtcolor(input_image, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()

  # cv2.imshow("Image", input_image)

  input_image_resized = cv2.resize(input_image (180,180))

  input_image_scaled = input_image_resized/255

  input_image_reshaped = np.reshape(input_image_scaled, [1,180,180,3])

  input_prediction = model1.predict(input_image_reshaped)

  print(input_prediction)
  if input_prediction > 0.5:
    print("Non-covid")
  else:
    print("covid")

## **Giving input as Covid scan image**

In [ ]:
# Calling a function
cov()

## **It classifies as covid**

## **Given input is Non-covid image**

In [ ]:
# Calling a function
cov()

## **It classifies as Non-covid**

### **Saving the model for further use**

In [ ]:
model1.save("Ct_Scan_Covid.h5")

In [ ]:
# Library to load the model
from tensorflow.keras.models import load_model

## **Loading the model to check the saved model**

In [ ]:
s_model = load_model1

## **Testing saved models that can predict real-world problems**

In [ ]:
import cv2
import matplotlib.pyplot as plt
def saved_model():

  input_image_path = input()
  input_image = cv2.imread(input_image_path)

  plt.imshow(cv2.cvtcolor(input_image, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()

  # cv2.imshow("Image", input_image)

  input_image_resized = cv2.resize(input_image (180,180))

  input_image_scaled = input_image_resized/255

  input_image_reshaped = np.reshape(input_image_scaled, [1,180,180,3])

  input_prediction = s_model.predict(input_image_reshaped)

  print(input_prediction)
  if input_prediction > 0.5:
    print("Non-covid")
  else:
    print("covid")

# **Giving input as Non-covid scan image**

In [ ]:
saved_model()

## **It classifies as Non-covid**

## **Finally our model working properly and classifies whether the CT Scan belongs to Covid or non covid**